In [1]:
import sys
sys.path.append('../')

%reload_ext autoreload
%autoreload 2
%matplotlib notebook
import matplotlib.pyplot as plt
import torch
import numpy as np
import utils.paramUtil as paramUtil
from utils.skeleton import Skeleton
from mylib import get_motion, animate3d, get_SMPL_layer, get6d, coco_bone, t2m_bone, smpl_smpl_bone
from utils.motion_process import recover_from_ric, recover_from_rot, recover_root_rot_pos
from pytorch3d.transforms import quaternion_to_axis_angle, matrix_to_axis_angle, rotation_6d_to_matrix, \
quaternion_to_matrix, axis_angle_to_quaternion, axis_angle_to_matrix, matrix_to_axis_angle, matrix_to_rotation_6d

Reading ../checkpoints/t2m/t2m_motiondiffuse/opt.txt


OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB (GPU 0; 23.65 GiB total capacity; 324.45 MiB already allocated; 11.31 MiB free; 362.00 MiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
data = torch.from_numpy(get_motion()).float()
joint_dataset = recover_from_ric(data, joints_num=22).numpy()
joint_w_hand = joint_dataset.copy()
joint_w_hand = np.concatenate((joint_w_hand, joint_w_hand[:, 20:22]), axis=1)
joint_w_hand.shape

In [ ]:
def get_neutral_angle_offset(_kinematic_tree, joints):
    _parents = [0] * len(joints)
    _parents[0] = -1
    for chain in _kinematic_tree:
        for j in range(1, len(chain)):
            _parents[chain[j]] = chain[j-1]
            
    angle_offset = torch.zeros(joints.shape)
    for i in range(1, angle_offset.shape[0]):
        angle_offset[i] = joints[i] - joints[_parents[i]]
        angle_offset[i] = angle_offset[i]/torch.linalg.norm(angle_offset[i] , ord=2)
    return angle_offset
pose_params = torch.zeros((1,72))
verts, Jtr = get_SMPL_layer(pose_params, display=True)
angle_offset = get_neutral_angle_offset(paramUtil.t2m_kinematic_chain, Jtr[0])

In [ ]:
def move_angle_to_parent(angle_old):
    _kinematic_tree = [[0, 2, 5, 8, 11], [0, 1, 4, 7, 10], [0, 3, 6, 9, 12, 15], [9, 14, 17, 19, 21,23], [9, 13, 16, 18, 20,22]]
    _parents = [0] * angle_old.shape[1]
    _parents[0] = -1
    for chain in _kinematic_tree:
        for j in range(1, len(chain)):
            _parents[chain[j]] = chain[j-1]
    angle_new = torch.zeros(angle_old.shape)
    for i in range(1, angle_new.shape[1]):
        angle_new[0,_parents[i]] = angle_old[0,i]
    return angle_new
a = move_angle_to_parent(pose_params)

### 1. Compare IK & FK
- [1] dataset j
- [2] dataset j --IK--> angle --FK--> j

In [ ]:
tgt_skel = Skeleton(angle_offset, paramUtil.t2m_kinematic_chain, 'cpu')
# 1. dataset j
vis_joint = joint_w_hand[0:1]

# 2.1 IK
quat_params = tgt_skel.inverse_kinematics_np(vis_joint, face_joint_idx=[2, 1, 17, 16])
tgt_offsets = tgt_skel.get_offsets_joints(torch.from_numpy(vis_joint[0]))
# 2.2 FK
joint_forward = tgt_skel.forward_kinematics(torch.from_numpy(quat_params).float(), root_pos=torch.from_numpy(vis_joint[0,0]))

joint_original_forward = np.concatenate((vis_joint[:,:22], joint_forward[:,:22]), axis=1)
animate3d(joint_original_forward, smpl_smpl_bone, first_total_standard=63)

### 1.1 Rest Pose
- [1] (0,0,0) --SMPL--> j
- [2] j --IK--> angle --FK--> j

In [6]:
tgt_skel = Skeleton(angle_offset, paramUtil.t2m_kinematic_chain, 'cpu')
# 1. SMPL layer (BLUE)
rest_pose = torch.zeros((1,24,3))
rest_pose[:,13] = torch.tensor((0,3.14/2,0))
rest_pose[:,14] = torch.tensor((0,-3.14/2,0))

verts, vis_joint = get_SMPL_layer(rest_pose.view(1,72), display=False)

# 2. [MD] FK  (RED)
rest_pose = torch.zeros((1,24,3))
rest_pose[:,13] = torch.tensor((0,3.14/2,0))
rest_pose[:,14] = torch.tensor((0,-3.14/2,0))


quat_params = axis_angle_to_quaternion(rest_pose)
tgt_offsets = tgt_skel.get_offsets_joints(vis_joint[0])
joint_forward = tgt_skel.forward_kinematics(quat_params, root_pos=vis_joint[0,0])

joint_original_forward = np.concatenate((vis_joint[:,:22], joint_forward[:,:22]), axis=1)
animate3d(joint_original_forward, smpl_smpl_bone, first_total_standard=63)

torch.Size([24, 6890])


### 2. Compare SMPL FK & original

In [7]:
tgt_skel = Skeleton(angle_offset, paramUtil.t2m_kinematic_chain, 'cpu')
vis_joint = joint_w_hand[0:1]
quat_params = tgt_skel.inverse_kinematics_np(vis_joint, face_joint_idx=[2, 1, 17, 16])

axis_angle = quaternion_to_axis_angle(torch.from_numpy(quat_params))
verts, joint_smpl = get_SMPL_layer(move_angle_to_parent(axis_angle), display=False)
joint_smpl[:, :, 1] -= joint_smpl[:, :, 1].min()
joint_original_smpl = np.concatenate((vis_joint[:,:22], joint_smpl[:,:22]), axis=1)
animate3d(joint_original_smpl, smpl_smpl_bone, first_total_standard=63)

torch.Size([24, 6890])


### DELETE ME

In [10]:
pose_params = torch.rand((1,72)) #torch.zeros((1,72))
verts, Jtr = get_SMPL_layer(pose_params, display=False)
animate3d(Jtr.numpy())

torch.Size([24, 6890])


In [ ]:
# import utils.paramUtil as paramUtil
from utils.skeleton import Skeleton

tgt_skel = Skeleton(torch.from_numpy(paramUtil.t2m_raw_offsets), paramUtil.t2m_kinematic_chain, 'cpu')
# 1. dataset j
vis_joint = joint[0:1]

# 2.1 IK
quat_params = tgt_skel.inverse_kinematics_np(vis_joint, face_joint_idx=[2, 1, 17, 16])
tgt_offsets = tgt_skel.get_offsets_joints(torch.from_numpy(vis_joint[0]))
# 2.2 FK
joint_forward = tgt_skel.forward_kinematics(torch.from_numpy(quat_params).float(), root_pos=torch.from_numpy(vis_joint[0,0]))

joint_original_forward = np.concatenate((vis_joint, joint_forward), axis=1)
smpl_smpl_bone = t2m_bone + (np.array(t2m_bone)+22).tolist()
animate3d(joint_original_forward, smpl_smpl_bone, first_total_standard=63)